In [ ]:
#!pip install --upgrade pip
#!pip install --upgrade setuptools
#!pip install GEOS

#!pip install GeobricksProj4ToEPSG
#!conda install -c conda-forge cartopy --yes
#!pip install geopandas
#print('ok')

In [ ]:
import os
import pandas
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from IPython.display import display, HTML
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import cascaded_union, unary_union#, union
import time
import io
#import rtree

os.chdir('/home/idies/workspace/Temporary/raddick/cra_scratch/')
print(os.getcwd())
outdir = '/home/idies/workspace/Storage/raddick/persistent/cra/'
print(outdir)

In [ ]:
baltimore_originators_df = pandas.read_csv('loans_processed.csv', encoding='utf-8', index_col='rownumber')
#baltimore_df.sample(3).T

print('{0:,.0f} loans nationwide, of all types.'.format(len(baltimore_originators_df)))
baltimore_originators_df = baltimore_originators_df[
    (baltimore_originators_df['state'] == 24) &    (baltimore_originators_df['county'] == 510) &
    (baltimore_originators_df['loan_type'] == 4) &
    (baltimore_originators_df['action_taken_type'] == 1)
]
print('{0:,.0f} small business loans originated to Baltimore.'.format(len(baltimore_originators_df)))

baltimore_originators_df_bk = baltimore_originators_df

#baltimore_originators_df.sample(3).T
print('\nDONE!')

In [ ]:
print('Reading from backup...')
baltimore_originators_df = baltimore_originators_df_bk

print('Totaling loan numbers and amounts...')
baltimore_originators_df = baltimore_originators_df.assign(nLoansTotal=baltimore_originators_df['nLoans1']+baltimore_originators_df['nLoans100k']+baltimore_originators_df['nLoans250k'])
baltimore_originators_df = baltimore_originators_df.assign(amtLoansTotal=baltimore_originators_df['amtLoans1']+baltimore_originators_df['amtLoans100k']+baltimore_originators_df['amtLoans250k'])
baltimore_originators_df = baltimore_originators_df.assign(amtLoansTotal_adjusted=baltimore_originators_df['amtLoans1_adjusted']+baltimore_originators_df['amtLoans100k_adjusted']+baltimore_originators_df['amtLoans250k_adjusted'])

print('Converting full level system to low/moderate/medium/upper...')
baltimore_originators_df = baltimore_originators_df.assign(cra_level = '')
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 1) & (x <=5))), 'cra_level'] = 'low'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 6) & (x <=8))), 'cra_level'] = 'moderate'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 9) & (x <=12))), 'cra_level'] = 'middle'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x == 13))), 'cra_level'] = 'upper'
baltimore_originators_df.loc[(baltimore_originators_df['income_group_total'].apply(lambda x: (x >= 14) & (x <=15))), 'cra_level'] = 'unknown'

#baltimore_originators_df.sample(3).T

baltimore_originators_df.groupby('cra_level').size().reindex(['low', 'moderate', 'middle', 'upper', 'unknown'])

In [ ]:
inst_year_df = baltimore_originators_df.groupby(['institution_name', 'activity_year'])['amtLoansTotal'].sum().unstack(1)#.sort_values)

sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
sortindex = pandas.Index(sortorder)
inst_year_df = inst_year_df.reindex(sortindex)

htmlstring = '<h1>Lending in Baltimore City by institution, 2004-2016</h1>'
htmlstring += '<table>'
htmlstring += '<tr>'
htmlstring += '<th>Institution</th>'
for i in range(2004, 2017):
    htmlstring += '<th>'+str(i)+'</th>'
htmlstring += '</tr>'
for idx, thisrow in inst_year_df.iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{0:}</td>'.format(idx)
    for i in range(0, len(thisrow)):
#    for i in range(5, 13):
        if ('<td>${0:,.0f}</td>'.format(thisrow.iloc[i]) == '<td>$nan</td>'):
            htmlstring += '<td></td>'
        else:
            htmlstring += '<td>${0:,.0f}</td>'.format(thisrow.iloc[i])
    htmlstring += '</tr>'
htmlstring += '</table>'


inst_year_df.to_csv(outdir+'institutions_by_year.csv', encoding='utf-8')
print('ok')

display(HTML(htmlstring))

In [ ]:
inst_year_df = baltimore_originators_df.groupby(['institution_name', 'activity_year'])['amtLoansTotal_adjusted'].sum().unstack(1)#.sort_values)

sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
sortindex = pandas.Index(sortorder)
inst_year_df = inst_year_df.reindex(sortindex)

htmlstring = '<h1>Lending in Baltimore City by institution, 2004-2016 (inflation-adjusted to 2016 dollars)</h1>'
htmlstring += '<table>'
htmlstring += '<tr>'
htmlstring += '<th>Institution</th>'
for i in range(2004, 2017):
    htmlstring += '<th>'+str(i)+'</th>'
htmlstring += '</tr>'
for idx, thisrow in inst_year_df.iterrows():
    htmlstring += '<tr>'
    htmlstring += '<td>{0:}</td>'.format(idx)
    for i in range(0, len(thisrow)):
        if ('<td>${0:,.0f}</td>'.format(thisrow.iloc[i]) == '<td>$nan</td>'):
            htmlstring += '<td></td>'
        else:
            htmlstring += '<td>${0:,.0f}</td>'.format(thisrow.iloc[i])
    htmlstring += '</tr>'
htmlstring += '</table>'


inst_year_df.to_csv(outdir+'institutions_by_year_adjusted.csv', encoding='utf-8')
print('OK')

display(HTML(htmlstring))

In [ ]:
def ymillions(x,pos):
    return '{0:.0f}'.format(x/1000000)

year_df = baltimore_originators_df.groupby(['activity_year'])['amtLoansTotal'].sum()#.unstack(1)
year_adj_df = baltimore_originators_df.groupby(['activity_year'])['amtLoansTotal_adjusted'].sum()#.unstack(1)

#sortorder = baltimore_originators_df[(baltimore_originators_df['institution_name'] != 'Unknown') & (baltimore_originators_df['activity_year'] == 2016)].groupby('institution_name', as_index=False)['amtLoansTotal'].sum().sort_values(by='amtLoansTotal', ascending=False)['institution_name'].values.tolist()
#sortindex = pandas.Index(sortorder)
#inst_year_df = inst_year_df.reindex(sortindex)

fig, ax1 = plt.subplots(figsize=(12,6))

ax1.plot(year_df.index, year_df.values, marker='o', label='Real')
ax1.plot(year_adj_df.index, year_adj_df.values, marker='o', label='Inflation-adjusted to 2016')

ax1.xaxis.set_major_locator(ticker.FixedLocator([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]))

ax1.set_ylim((0,600000000))
ax1.yaxis.set_major_locator(ticker.AutoLocator())
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(ymillions))

plt.tick_params(axis='both', which='major', labelsize=12)
plt.xlabel('Year', fontsize=16)
plt.ylabel('Total loans (millions)', fontsize=16)
plt.title('Total CRA loans in Baltimore City', fontsize=18)
plt.legend(fontsize=12)
plt.show()

# Loans by geography

In [ ]:
baltimore_tracts_df = pandas.read_csv('tracts_processed.csv', encoding='utf-8', index_col='rownumber')
print('{0:,.0f} tract datas nationwide.'.format(len(baltimore_tracts_df)))

baltimore_tracts_df = baltimore_tracts_df[(baltimore_tracts_df['state'] == 24) & (baltimore_tracts_df['county'] == 510)]
print('{0:,.0f} tract datas in Baltimore.'.format(len(baltimore_tracts_df)))

print('Fixing MSA weirdness...')
baltimore_tracts_df = baltimore_tracts_df.drop('msa_name', axis=1)
baltimore_tracts_df = baltimore_tracts_df.rename(columns={'cbsa_name': 'msa_name'})

print('Backing up...')
baltimore_tracts_df_bk = baltimore_tracts_df

print('\nDONE!')


In [ ]:
print('Retrieving from backup...')
baltimore_tracts_df = baltimore_tracts_df_bk


print('Converting full level system to low/moderate/medium/upper...')
baltimore_tracts_df = baltimore_tracts_df.assign(cra_level = '')
baltimore_tracts_df.loc[(baltimore_tracts_df['income_group_total'].apply(lambda x: (x >= 1) & (x <=5))), 'cra_level'] = 'low'
baltimore_tracts_df.loc[(baltimore_tracts_df['income_group_total'].apply(lambda x: (x >= 6) & (x <=8))), 'cra_level'] = 'moderate'
baltimore_tracts_df.loc[(baltimore_tracts_df['income_group_total'].apply(lambda x: (x >= 9) & (x <=12))), 'cra_level'] = 'middle'
baltimore_tracts_df.loc[(baltimore_tracts_df['income_group_total'].apply(lambda x: (x == 13))), 'cra_level'] = 'upper'
baltimore_tracts_df.loc[(baltimore_tracts_df['income_group_total'].apply(lambda x: (x >= 14) & (x <=15))), 'cra_level'] = 'unknown'

print('Backing up...')
baltimore_tracts_df_bk = baltimore_tracts_df

baltimore_tracts_df.sample(3).T


In [ ]:
print('retrieving from backup...')
baltimore_tracts_df = baltimore_tracts_df_bk

print('grouping...')

tractdata = baltimore_tracts_df[baltimore_tracts_df['loan_indicator'] == 'Y'].groupby(['census_tract', 'cra_level', 'activity_year', 'institution_name']).size()

tractdata = tractdata.sort_index()
# Run this to get every index value to display in every row:
#pandas.set_option('display.multi_sparse', False)

# GET ALL ROWS THAT MATCH A GIVEN CRITERION FROM A GIVEN LEVEL
#tractdata.xs(101.00, level=0) #Access all loans given in census tract 101
#tractdata.xs('low', level=1) # Access all loans given to low income group 
#tractdata.xs(2016, level=2) # Access all loans given in 2016
#tractdata.xs('WELLS FARGO BANK, N.A. (CA)', level=3) # Access all loans given by Wells Fargo

# FIND NUMBER OF LOANS FOR EACH VALUE AT EACH LEVEL

#tractdata.groupby([pandas.Grouper(level='census_tract')]).sum() # Number of loans given in each census tract
#tractdata.groupby([pandas.Grouper(level='cra_level')]).sum() # Number of loans given at each CRA income level
#tractdata.groupby([pandas.Grouper(level='activity_year')]).sum() # Number of loans given per year
#tractdata.groupby([pandas.Grouper(level='institution_name')]).sum() # Number of loans given per institution


# ITERATE THROUGH VALUES OF AN INDEX LEVEL AND SEE THE GROUPS FORMED BY THOSE VALUES
#grouped_by_census_tract = tractdata.groupby('census_tract')
#for name, group in grouped_by_census_tract:
#    print(name)
#    print(group)
#    print('\n')

# ACCESS A GROUP ASSOCIATED WITH A SPECIFIC VALUE ONE ONE LEVEL...
#grouped_by_census_tract = tractdata.groupby('census_tract')
#grouped_by_census_tract.get_group(101.00)
# OR ON MULTIPLE LEVELS...
#tractdata.groupby(['census_tract', 'institution_name']).get_group((101, '1ST MARINER BANK (MD)')) # note get_group takes a tuple

print('backing up...')
tractdata_bk = tractdata

print('OK')
#tractdata

In [ ]:
print('getting from backup...')
tractdata = tractdata_bk
#tractdata
grouped_by_census_tract = tractdata.groupby('census_tract').sum()
print('backing up...')
grouped_by_census_tract_bk = grouped_by_census_tract
print('ok')

In [ ]:
s = time.time()
print('reading shapefile...')

tract_shapes_gdf = gpd.read_file('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/census_tracts_2010/geo_export_c50bbe56-543e-4878-9c9f-c56be327600a.shp', encoding='utf-8')
tract_shapes_gdf = tract_shapes_gdf.assign(tractname = pandas.to_numeric(tract_shapes_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
tract_shapes_gdf = tract_shapes_gdf.set_index('tractname')
e = time.time()

print('Read {0:,.0f} Maryland census tracts in {1:.3f} seconds.'.format(len(tract_shapes_gdf), e-s))

print('Calculating percent black...')
tract_shapes_gdf = tract_shapes_gdf.assign(percent_black = tract_shapes_gdf['blk_afam'] / tract_shapes_gdf['population'])


print('Reading loan by tract data from backup...')
grouped_by_census_tract = grouped_by_census_tract_bk

tract_shapes_gdf = tract_shapes_gdf.assign(nLoans=grouped_by_census_tract)

print('backing up...')
tract_shapes_gdf_bk = tract_shapes_gdf

print('ok')

In [ ]:
print('reading water features...')
water_gdf = gpd.read_file('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/water/water.shp', encoding='utf-8')
water_gdf = water_gdf.set_index('OBJECTID')
water_gdf.sort_values('Shape_Area', ascending=False)
#water_gdf.crs

print('matching up CRSes...')
tract_shapes_gdf = tract_shapes_gdf.to_crs(water_gdf.crs)
print('ok')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(60,34))

ax1 = tract_shapes_gdf.plot(ax=ax1, column='percent_black', cmap='Greys', edgecolor='yellow')
ax1.set_title('Percent black residents', fontsize=36)
ax1.set_aspect('equal')
water_gdf.plot(ax=ax1)

for ix, thisrow in tract_shapes_gdf[tract_shapes_gdf['population'].notnull()].iterrows():
    annotator = str(ix)
    if (thisrow['population'] > 0):
        annotator += '\n' + '{0:,.0%}'.format(thisrow['percent_black'])
        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax1.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax2 = tract_shapes_gdf.plot(ax=ax2, column='nLoans', cmap='viridis', edgecolor='black')
ax2.set_title('Number of CRA loans originated', fontsize=36)
ax2.set_aspect('equal')
water_gdf.plot(ax=ax2, color='w')

for ix, thisrow in tract_shapes_gdf[tract_shapes_gdf['population'].notnull()].iterrows():
    annotator = str(ix)
    if (thisrow['population'] > 0):
        annotator += '\n' + '{0:,.0f}'.format(thisrow['nLoans'])
#        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax2.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax1.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
ax2.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

# add colorbar
cax1 = fig.add_axes([0.12, 0.08, 0.36, 0.03])
sm1 = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=36)
cbar1.set_label('Percent black', fontsize=42)

# add colorbar
cax2 = fig.add_axes([0.545, 0.08, 0.36, 0.03])
sm2 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=tract_shapes_gdf['nLoans'].max()))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, orientation='horizontal')# ticks=np.arange(0, 110, 10), orientation='horizontal')

cax2.tick_params(labelsize=36)
cbar2.set_label('Number of CRA loans', fontsize=42)

#plt.show()

print('Saving...')
fig.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_census_tracts.svg', format='svg', dpi=300)
print('DONE!')


In [ ]:
print('retrieving from backup...')
baltimore_tracts_df = baltimore_tracts_df_bk
#baltimore_tracts_df

print('adding CBSA data...')
census_tract_to_neighborhood_df = pandas.read_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/census_tract_to_neighborhood.csv', low_memory=False)#, index_col='NAME10')
census_tract_to_neighborhood_df['NAME10'] = pandas.to_numeric(census_tract_to_neighborhood_df['NAME10'], errors='coerce')
census_tract_to_neighborhood_df = census_tract_to_neighborhood_df.set_index('NAME10')

baltimore_tracts_df = baltimore_tracts_df.join(census_tract_to_neighborhood_df, how='left', on='census_tract')
#baltimore_tracts_df

print('backing up...')
baltimore_tracts_df_bk = baltimore_tracts_df
print('ok')

In [ ]:
print('retrieving from backup...')
baltimore_tracts_df = baltimore_tracts_df_bk

print('grouping by neighborhoood...')

csa_data = baltimore_tracts_df[baltimore_tracts_df['loan_indicator'] == 'Y'].groupby(['CSA2010', 'cra_level', 'activity_year', 'institution_name']).size()

csa_data = csa_data.sort_index()
# Run this to get every index value to display in every row:
#pandas.set_option('display.multi_sparse', False)

# GET ALL ROWS THAT MATCH A GIVEN CRITERION FROM A GIVEN LEVEL
#tractdata.xs(101.00, level=0) #Access all loans given in census tract 101
#tractdata.xs('low', level=1) # Access all loans given to low income group 
#tractdata.xs(2016, level=2) # Access all loans given in 2016
#tractdata.xs('WELLS FARGO BANK, N.A. (CA)', level=3) # Access all loans given by Wells Fargo

# FIND NUMBER OF LOANS FOR EACH VALUE AT EACH LEVEL

#tractdata.groupby([pandas.Grouper(level='census_tract')]).sum() # Number of loans given in each census tract
#tractdata.groupby([pandas.Grouper(level='cra_level')]).sum() # Number of loans given at each CRA income level
#tractdata.groupby([pandas.Grouper(level='activity_year')]).sum() # Number of loans given per year
#tractdata.groupby([pandas.Grouper(level='institution_name')]).sum() # Number of loans given per institution


# ITERATE THROUGH VALUES OF AN INDEX LEVEL AND SEE THE GROUPS FORMED BY THOSE VALUES
#grouped_by_census_tract = tractdata.groupby('census_tract')
#for name, group in grouped_by_census_tract:
#    print(name)
#    print(group)
#    print('\n')

# ACCESS A GROUP ASSOCIATED WITH A SPECIFIC VALUE ONE ONE LEVEL...
#grouped_by_census_tract = tractdata.groupby('census_tract')
#grouped_by_census_tract.get_group(101.00)
# OR ON MULTIPLE LEVELS...
#tractdata.groupby(['census_tract', 'institution_name']).get_group((101, '1ST MARINER BANK (MD)')) # note get_group takes a tuple

#print('backing up...')
#tractdata_bk = tractdata

#print('OK')
csa_data
#tractdata

In [ ]:
grouped_by_csa = csa_data.groupby('CSA2010').sum()
grouped_by_csa.name = 'nLoans'
grouped_by_csa

In [ ]:
s = time.time()
print('reading shapefile...')

csa_shapes_gdf = gpd.read_file('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/csa/community_statistical_area.shp', encoding='utf-8')
csa_shapes_gdf = csa_shapes_gdf.set_index('DC55NEIG')                            
e = time.time()

print('Read {0:,.0f} Maryland census tracts in {1:.3f} seconds.'.format(len(tract_shapes_gdf), e-s))

#print('Calculating percent black...')
#tract_shapes_gdf = tract_shapes_gdf.assign(percent_black = tract_shapes_gdf['blk_afam'] / tract_shapes_gdf['population'])


#print('Reading loan by tract data from backup...')
#grouped_by_census_tract = grouped_by_census_tract_bk

#tract_shapes_gdf = tract_shapes_gdf.assign(nLoans=grouped_by_census_tract)

#print('backing up...')
#tract_shapes_gdf_bk = tract_shapes_gdf


csa_shapes_gdf = csa_shapes_gdf[csa_shapes_gdf['NEIG55ID'] != 91]
#print('ok')
csa_shapes_gdf = csa_shapes_gdf.join(grouped_by_csa, how='left')


csa_shapes_gdf


In [ ]:
print('reading water features...')
water_gdf = gpd.read_file('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/water/water.shp', encoding='utf-8')
water_gdf = water_gdf.set_index('OBJECTID')
water_gdf.sort_values('Shape_Area', ascending=False)
#water_gdf.crs

print('matching up CRSes...')
csa_shapes_gdf = csa_shapes_gdf.to_crs(water_gdf.crs)
print('ok')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(60,34))

ax1 = tract_shapes_gdf.plot(ax=ax1, column='percent_black', cmap='Greys', edgecolor='yellow')
ax1.set_title('Percent black residents', fontsize=36)
ax1.set_aspect('equal')
water_gdf.plot(ax=ax1)

for ix, thisrow in tract_shapes_gdf[tract_shapes_gdf['population'].notnull()].iterrows():
    annotator = str(ix)
    if (thisrow['population'] > 0):
        annotator += '\n' + '{0:,.0%}'.format(thisrow['percent_black'])
        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax1.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax2 = csa_shapes_gdf.plot(ax=ax2, column='nLoans', cmap='viridis', edgecolor='black')
ax2.set_title('Number of CRA loans originated', fontsize=36)
ax2.set_aspect('equal')
water_gdf.plot(ax=ax2, color='w')

#for ix, thisrow in csa_shapes_gdf[csa_shapes_gdf['population'].notnull()].iterrows():
for ix, thisrow in csa_shapes_gdf.iterrows():
    annotator = str(ix)
#    if (thisrow['population'] > 0):
    annotator += '\n' + '{0:,.0f}'.format(thisrow['nLoans'])
#        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax2.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax1.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
ax2.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

# add colorbar
cax1 = fig.add_axes([0.12, 0.08, 0.36, 0.03])
sm1 = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=36)
cbar1.set_label('Percent black', fontsize=42)

# add colorbar
cax2 = fig.add_axes([0.545, 0.08, 0.36, 0.03])
sm2 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=csa_shapes_gdf['nLoans'].max()))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, orientation='horizontal')# ticks=np.arange(0, 110, 10), orientation='horizontal')

cax2.tick_params(labelsize=36)
cbar2.set_label('Number of CRA loans', fontsize=42)

#plt.show()

print('Saving...')
fig.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_census_tracts.svg', format='svg', dpi=300)
print('DONE!')


In [ ]:
tract_list = baltimore_tracts_df['census_tract'][baltimore_tracts_df['loan_indicator'] == 'Y'].drop_duplicates().sort_values().tolist()

tract_guide_df = pandas.DataFrame(data=tract_list, columns=['census_tract'])
tract_guide_df = tract_guide_df.set_index('census_tract')

tract_guide_df = tract_guide_df.assign(nLoans = 0, 
                                       nLoansLow = 0, 
                                       nLoansModerate = 0, 
                                       nLoansMiddle = 0, 
                                       nLoansUpper = 0, 
                                       nLoansUnknown = 0)

for tractnum, thisrow in tract_guide_df.iterrows():
    tract_guide_df['nLoans'].loc[tractnum] = len(baltimore_tracts_df[baltimore_tracts_df['census_tract'] == tractnum])
    tract_guide_df['nLoansLow'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'low')])
    tract_guide_df['nLoansModerate'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'moderate')])
    tract_guide_df['nLoansMiddle'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'middle')])
    tract_guide_df['nLoansUpper'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'upper')])
    tract_guide_df['nLoansUnknown'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'unknown')])
    
    
#for thistract in tract_list:
column_order = ['nLoans', 'nLoansLow', 'nLoansModerate', 'nLoansMiddle', 'nLoansUpper', 'nLoansUnknown']
tract_guide_df = tract_guide_df[column_order]

print('Backing up...')
tract_guide_df_bk = tract_guide_df

tract_guide_df.sample(1)


In [ ]:

#loans_by_neighborhood = pandas.Series(
#    data=baltimore_tracts_df[baltimore_tracts_df['loan_indicator'] == 'Y'].groupby(['neighborhood']).size(), 
#    name='nLoans')

#loans_by_neighborhood.to_csv('loans_by_neighborhood.csv', index=True, header=True)
#print('ok')
#loans_by_neighborhood

#baltimore_tracts_df
#loans_by_neighborhood

#print('Found {0:,.0f} tracts in Baltimore City; {1:,.0f} received loans.'.format(len(baltimore_tracts_df), len(baltimore_tracts_df[baltimore_tracts_df['loan_indicator'] == 'Y'])))
#baltimore_tracts_df[['institution_name', 'activity_year']].sample(3).T
#baltimore_tracts_df.groupby(['state', 'county']).size() # All are correctly state=24 county=510
#baltimore_tracts_df.groupby(['msa', 'cbsa_name']).size() # All are correctly msa = 12580, cbsa_name = 'Baltimore-Columbia-Towson, MD'

print('\n{0:,.0f} census tracts received loans, across {1:,.0f} neighborhoods.'.format(
    len(baltimore_tracts_df['census_tract'][
        (baltimore_tracts_df['loan_indicator'] == 'Y') &
        (baltimore_tracts_df['census_tract'].notnull())
    ].drop_duplicates()
       ),
    len(baltimore_tracts_df['neighborhood'][
        (baltimore_tracts_df['loan_indicator'] == 'Y') &
        (baltimore_tracts_df['neighborhood'].notnull())
    ].drop_duplicates()
       )
))



tract_list = baltimore_tracts_df['census_tract'][baltimore_tracts_df['loan_indicator'] == 'Y'].drop_duplicates().sort_values().tolist()
#tract_list

neighborhood_list = baltimore_tracts_df['neighborhood'][
    (baltimore_tracts_df['loan_indicator'] == 'Y') &
    (baltimore_tracts_df['neighborhood'].notnull())
].drop_duplicates().sort_values().tolist()
#neighborhood_list

print('\nDONE!')


In [ ]:

tract_guide_df = pandas.DataFrame(data=tract_list, columns=['census_tract'])
tract_guide_df = tract_guide_df.set_index('census_tract')

tract_guide_df = tract_guide_df.assign(nLoans = 0, 
                                       nLoansLow = 0, 
                                       nLoansModerate = 0, 
                                       nLoansMiddle = 0, 
                                       nLoansUpper = 0, 
                                       nLoansUnknown = 0)

for tractnum, thisrow in tract_guide_df.iterrows():
    tract_guide_df['nLoans'].loc[tractnum] = len(baltimore_tracts_df[baltimore_tracts_df['census_tract'] == tractnum])
    tract_guide_df['nLoansLow'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'low')])
    tract_guide_df['nLoansModerate'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'moderate')])
    tract_guide_df['nLoansMiddle'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'middle')])
    tract_guide_df['nLoansUpper'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'upper')])
    tract_guide_df['nLoansUnknown'].loc[tractnum] = len(baltimore_tracts_df[(baltimore_tracts_df['census_tract'] == tractnum) & (baltimore_tracts_df['cra_level'] == 'unknown')])
    
    
#for thistract in tract_list:
column_order = ['nLoans', 'nLoansLow', 'nLoansModerate', 'nLoansMiddle', 'nLoansUpper', 'nLoansUnknown']
tract_guide_df = tract_guide_df[column_order]

print('Backing up...')
tract_guide_df_bk = tract_guide_df

tract_guide_df.sample(1)


In [ ]:
s = time.time()
print('reading shapefile...')

tracts_gdf = gpd.read_file('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/census_tracts_2010/geo_export_c50bbe56-543e-4878-9c9f-c56be327600a.shp', encoding='utf-8')
tracts_gdf = tracts_gdf.assign(tractname = pandas.to_numeric(tracts_gdf['name'].apply(lambda x: x.split(' ')[-1]), errors='coerce'))
tracts_gdf = tracts_gdf.set_index('tractname')
e = time.time()

print('Read {0:,.0f} Maryland census tracts in {1:.3f} seconds.'.format(len(tracts_gdf), e-s))


print('Reading loan by tract data from backup...')
tract_guide_df = tract_guide_df_bk

tract_guide_df = tract_guide_df.join(tracts_gdf, how='left')

tract_guide_df = tract_guide_df.assign(percent_black = (tract_guide_df['blk_afam'] / tract_guide_df['population']))

tract_guide_gdf = gpd.GeoDataFrame(data=tract_guide_df, crs=tracts_gdf.crs, geometry=tracts_gdf['geometry'])

#tract_guide_gdf.sample(1)
print('backing up GeoDataFrame...')
tract_guide_gdf_bk = tract_guide_gdf
print('OK')
#tract_guide_gdf.sample(1).T


In [ ]:
print('reading from backup...')
tract_guide_gdf = tract_guide_gdf_bk

fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(60,34))

ax1 = tract_guide_gdf.plot(ax=ax1, column='percent_black', cmap='Greys', edgecolor='yellow')
ax1.set_title('Percent black residents', fontsize=36)
ax1.set_aspect('equal')

for ix, thisrow in tract_guide_gdf[tract_guide_gdf['population'].notnull()].iterrows():
    annotator = str(ix)
    if (thisrow['population'] > 0):
        annotator += '\n' + '{0:,.0%}'.format(thisrow['percent_black'])
        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax1.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax2 = tract_guide_gdf.plot(ax=ax2, column='nLoans', cmap='viridis', edgecolor='black')
ax2.set_title('Number of CRA loans originated', fontsize=36)
ax2.set_aspect('equal')

for ix, thisrow in tract_guide_gdf[tract_guide_gdf['population'].notnull()].iterrows():
    annotator = str(ix)
    if (thisrow['population'] > 0):
        annotator += '\n' + '{0:,.0f}'.format(thisrow['nLoans'])
#        annotator += '\n' + '(n = {0:,.0f})'.format(thisrow['population'])
    ax2.annotate(annotator, 
                xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                ha='center', va='center', backgroundcolor='white', fontsize=11)


ax1.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
ax2.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

# add colorbar
cax1 = fig.add_axes([0.12, 0.08, 0.36, 0.03])
sm1 = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=36)
cbar1.set_label('Percent black', fontsize=42)

# add colorbar
cax2 = fig.add_axes([0.545, 0.08, 0.36, 0.03])
sm2 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=tract_guide_gdf['nLoans'].max()))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, format='%.0f', orientation='horizontal')# ticks=np.arange(0, 110, 10), orientation='horizontal')

cax2.tick_params(labelsize=36)
cbar2.set_label('Number of CRA loans', fontsize=42)

#plt.show()

print('Saving...')
fig.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_census_tracts.svg', format='svg', dpi=300)
print('DONE!')


In [ ]:
neighborhood_guide_df = pandas.DataFrame(data=neighborhood_list, columns=['neighborhood'])
neighborhood_guide_df = neighborhood_guide_df.set_index('neighborhood')

neighborhood_guide_df = neighborhood_guide_df.assign(nLoans = 0, 
                                       nLoansLow = 0, 
                                       nLoansModerate = 0, 
                                       nLoansMiddle = 0, 
                                       nLoansUpper = 0, 
                                       nLoansUnknown = 0)

for nh, thisrow in neighborhood_guide_df.iterrows():
    neighborhood_guide_df['nLoans'].loc[nh] = len(baltimore_tracts_df[baltimore_tracts_df['neighborhood'] == nh])
    neighborhood_guide_df['nLoansLow'].loc[nh] = len(baltimore_tracts_df[(baltimore_tracts_df['neighborhood'] == nh) & (baltimore_tracts_df['cra_level'] == 'low')])
    neighborhood_guide_df['nLoansModerate'].loc[nh] = len(baltimore_tracts_df[(baltimore_tracts_df['neighborhood'] == nh) & (baltimore_tracts_df['cra_level'] == 'moderate')])
    neighborhood_guide_df['nLoansMiddle'].loc[nh] = len(baltimore_tracts_df[(baltimore_tracts_df['neighborhood'] == nh) & (baltimore_tracts_df['cra_level'] == 'middle')])
    neighborhood_guide_df['nLoansUpper'].loc[nh] = len(baltimore_tracts_df[(baltimore_tracts_df['neighborhood'] == nh) & (baltimore_tracts_df['cra_level'] == 'upper')])
    neighborhood_guide_df['nLoansUnknown'].loc[nh] = len(baltimore_tracts_df[(baltimore_tracts_df['neighborhood'] == nh) & (baltimore_tracts_df['cra_level'] == 'unknown')])
    

column_order = ['nLoans', 'nLoansLow', 'nLoansModerate', 'nLoansMiddle', 'nLoansUpper', 'nLoansUnknown']
neighborhood_guide_df = neighborhood_guide_df[column_order]
neighborhood_guide_df#[neighborhood_guide_df['nLoans'] == neighborhood_guide_df['nLoansLow']]


In [ ]:
nfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/Neighborhoods.shp'
ngeo = gpd.read_file(nfile)
print('Geographies read')
#ngeo = ngeo.set_index('Name')
ngeo = ngeo.assign(percent_white=100*(ngeo['White']/ngeo['Population']))
ngeo = ngeo.assign(percent_black=100*(ngeo['Blk_AfAm']/ngeo['Population']))

ngeo.assign(big_neighborhood='')

ngeo.loc[ngeo['Name'].isin(['Canton', 'Cherry Hill', 'Edmondson Village', 'Highlandtown', 'Lauraville', 'Loch Raven', 'South Baltimore']), 'big_neighborhood'] = ngeo['Name']

ngeo.loc[ngeo['Name'].isin(['Allendale', 'Irvington', 'Carroll-South Hilton']), 'big_neighborhood'] = 'Allendale/Irvington/S. Hilton'
ngeo.loc[ngeo['Name'].isin(['Beechfield', 'Ten Hills', 'West Hills']), 'big_neighborhood'] = 'Beechfield/Ten Hills/West Hills'
ngeo.loc[ngeo['Name'].isin(['Brooklyn', 'Curtis Bay', 'Hawkins Point']), 'big_neighborhood'] = 'Brooklyn/Curtis Bay/Hawkins Point'
ngeo.loc[ngeo['Name'].isin(['Cedonia', 'Frankford']), 'big_neighborhood'] = 'Cedonia/Frankford'
ngeo.loc[ngeo['Name'].isin(['Chinquapin Park', 'Belvedere']), 'big_neighborhood'] = 'Chinquapin Park/Belvedere'
ngeo.loc[ngeo['Name'].isin(['Berea', 'Clifton Park', 'South Clifton Park']), 'big_neighborhood'] = 'Clifton-Berea'
ngeo.loc[ngeo['Name'].isin(['Cross Country', 'Cheswolde']), 'big_neighborhood'] = 'Cross-Country/Cheswolde'
ngeo.loc[ngeo['Name'].isin(['Dickeyville', 'Franklintown']), 'big_neighborhood'] = 'Dickeyville/Franklintown'
ngeo.loc[ngeo['Name'].isin(['Dorchester', 'Ashburton']), 'big_neighborhood'] = 'Dorchester/Ashburton'
ngeo.loc[ngeo['Name'].isin(['Downtown', 'Downtown West' 'Seton Hill']), 'big_neighborhood'] = 'Downtown/Seton Hill'
ngeo.loc[ngeo['Name'].isin(['Fells Point', 'Upper Fells Point']), 'big_neighborhood'] = 'Fells Point'
ngeo.loc[ngeo['Name'].isin(['Central Forest Park', 'Concerned Citizens Of Forest Park', 'Forest Park', 'Forest Park Golf Course', 'West Forest Park', 'Walbrook']), 'big_neighborhood'] = 'Forest Park/Walbrook'
ngeo.loc[ngeo['Name'].isin(['Glen', 'Fallstaff']), 'big_neighborhood'] = 'Glen-Fallstaff'
ngeo.loc[ngeo['Name'].isin(['Charles Village', 'Barclay']), 'big_neighborhood'] = 'Greater Charles Village/Barclay'
ngeo.loc[ngeo['Name'].isin(['Mid-Govans', 'Winston-Govans']), 'big_neighborhood'] = 'Greater Govans'
ngeo.loc[ngeo['Name'].isin(['Mondawmin']), 'big_neighborhood'] = 'Greater Mondawmin'
ngeo.loc[ngeo['Name'].isin(['Roland Park', 'North Roland Park/Poplar Hill']), 'big_neighborhood'] = 'Greater Roland Park/Poplar Hill'
ngeo.loc[ngeo['Name'].isin(['Rosemont', 'Rosemont East', 'Rosemont Homeowners/Tenants']), 'big_neighborhood'] = 'Greater Rosemont'
ngeo.loc[ngeo['Name'].isin(['Hamilton Hills']), 'big_neighborhood'] = 'Hamilton'
ngeo.loc[ngeo['Name'].isin(['Little Italy']), 'big_neighborhood'] = 'Harbor East/Little Italy'
ngeo.loc[ngeo['Name'].isin(['North Harford Road']), 'big_neighborhood'] = 'Harford/Echodale'
ngeo.loc[ngeo['Name'].isin(['Howard Park', 'West Arlington']), 'big_neighborhood'] = 'Howard Park/West Arlington'
ngeo.loc[ngeo['Name'].isin(['Inner Harbor', 'Federal Hill']), 'big_neighborhood'] = 'Inner Harbor/Federal Hill'
ngeo.loc[ngeo['Name'].isin(['Madison-Eastend']), 'big_neighborhood'] = 'Madison/East End'
ngeo.loc[ngeo['Name'].isin(['Medfield', 'Hampden', 'Woodberry', 'Remington']), 'big_neighborhood'] = 'Medfield/Hampden/Woodberry/Remington'
ngeo.loc[ngeo['Name'].isin(['Mid-Town Belvedere', 'Midtown-Edmondson']), 'big_neighborhood'] = 'Midtown'
ngeo.loc[ngeo['Name'].isin(['East Baltimore Midway', 'Coldstream Homestead Montebello']), 'big_neighborhood'] = 'Midway/Coldstream'
ngeo.loc[ngeo['Name'].isin(['Morrell Park', 'Violetville']), 'big_neighborhood'] = 'Morrell Park/Violetville'
ngeo.loc[ngeo['Name'].isin(['Mount Washington', 'Coldspring']), 'big_neighborhood'] = 'Mount Washington/Coldspring'
ngeo.loc[ngeo['Name'].isin(['Guilford', 'Homeland', 'Villages Of Homeland', 'York-Homeland']), 'big_neighborhood'] = 'North Baltimore/Guilford/Homeland'
ngeo.loc[ngeo['Name'].isin(['New Northwood', 'Original Northwood']), 'big_neighborhood'] = 'Northwood'
ngeo.loc[ngeo['Name'].isin(['Oldtown', 'Middle East']), 'big_neighborhood'] = 'Oldtown/Middle East'
ngeo.loc[ngeo['Name'].isin(['Orangeville']), 'big_neighborhood'] = 'Orangeville/East Highlandtown'
ngeo.loc[ngeo['Name'].isin(['Patterson Park Neighborhood', 'Patterson Park']), 'big_neighborhood'] = 'Patterson Park North & East'
ngeo.loc[ngeo['Name'].isin(['Penn North', 'Reservoir Hill']), 'big_neighborhood'] = 'Penn North/Reservoir Hill'
ngeo.loc[ngeo['Name'].isin(['Pimlico Good Neighbors', 'Arlington', 'East Arlington', 'West Arlington']), 'big_neighborhood'] = 'Pimlico/Arlington/Hilltop'
ngeo.loc[ngeo['Name'].isin(['Pimlico Good Neighbors', 'Arlington', 'East Arlington', 'West Arlington']), 'big_neighborhood'] = 'Pimlico/Arlington/Hilltop'
ngeo.loc[ngeo['Name'].isin(['Poppleton', 'Hollins Market']), 'big_neighborhood'] = 'Poppleton/The Terraces/Hollins Market'
ngeo.loc[ngeo['Name'].isin(['Poppleton', 'Hollins Market']), 'big_neighborhood'] = 'Poppleton/The Terraces/Hollins Market'
ngeo.loc[ngeo['Name'].isin(['Sandtown-Winchester', 'Harlem Park']), 'big_neighborhood'] = 'Sandtown-Winchester/Harlem Park'
ngeo.loc[ngeo['Name'].isin(['Better Waverly', 'Waverly']), 'big_neighborhood'] = 'The Waverlies'
ngeo.loc[ngeo['Name'].isin(['Upton', 'Druid Heights']), 'big_neighborhood'] = 'Upton/Druid Heights'
ngeo.loc[ngeo['Name'].isin(['Westport', 'Mount Winans', 'Lakeland']), 'big_neighborhood'] = 'Westport/Mt. Winans/Lakeland'

ngeo = ngeo.fillna('')

#ngeo['big_neighborhood'].drop_duplicates().sort_values()
#ngeo['Name'][ngeo['Name'].apply(lambda x: 'lake' in x.lower())]

#neighborhood_guide_df.head(2)
ngeo.sample(1)
#ngeo['geodataEGI']
#len(ngeo)

In [ ]:
# Find neighborhoods that still need to be matched to bigger neighborhoods
#ngeo['Name'][ngeo['big_neighborhood'] == ''].sort_values().to_csv('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/mystery_neighborhoods.csv', encoding='utf-8')
#print('ok')
neighborhood_guide_df_bk = neighborhood_guide_df
print('backed up')

In [ ]:
#neighborhood_guide_df.reset_index().merge(ngeo, )
#ngeo.columns
#guide_neighborhoods_list = neighborhood_guide_df.index.values.tolist()
neighborhood_guide_df = neighborhood_guide_df_bk
print('retrieved from backup')

s = time.time()
columns_to_agg = ngeo.columns.tolist()
do_not_agg_list = ['Name', 'OBJECTID', 'COLOR_2', 'geodataEGI', 'Pop_dens', 'AveHHsize']
do_not_agg_list += ['created_us', 'created_da', 'last_edite', 'last_edi_1']
do_not_agg_list += ['ShapeSTAre', 'ShapeSTLen', 'geometry']
do_not_agg_list += ['percent_white', 'percent_black', 'big_neighborhood']
for x in do_not_agg_list:
    columns_to_agg.remove(x)

new_columns_df = pandas.DataFrame(data=None, columns=columns_to_agg, index=neighborhood_guide_df.index)
neighborhood_guide_df = pandas.concat([neighborhood_guide_df, new_columns_df], axis=1)

for thisnh, thisrow in neighborhood_guide_df.iterrows():
    print(thisnh+'...')
    for thiscol in columns_to_agg:
        #print(neighborhood_guide_df[thiscol].loc[thisnh])
#        z = 
        #print('{0}: {1:,.0f}'.format(thiscol, z))
        neighborhood_guide_df.loc[thisnh, thiscol] = ngeo[thiscol][ngeo['big_neighborhood'] == thisnh].sum()
        #print(ngeo.groupby('big_neighborhood')[thiscol].sum().values[0])
    #print('\n')
#        neighborhood_guide_df.loc[thisnh, thiscol] = ngeo.groupby('big_neighborhood')[thiscol].sum()#ngeo[ngeo['big_neighborhood'] == thisnh][thiscol].sum()
#        print('\n')
#    
    #print('\n')

print('Got all {0:,.0f} big neighborhoods!'.format(len(neighborhood_guide_df)))

print('adding percent black...')
neighborhood_guide_df = neighborhood_guide_df.assign(percent_black=100*(neighborhood_guide_df['Blk_AfAm'][neighborhood_guide_df['Population'] > 0] / neighborhood_guide_df['Population'][neighborhood_guide_df['Population'] > 0]))
e = time.time()

print('backing up...')
neighborhood_guide_df_bk = neighborhood_guide_df

print('\nDONE in {0:0f} seconds.'.format(e-s))


In [ ]:
print('retrieving from backup...')
neighborhood_guide_df = neighborhood_guide_df_bk

thecrs = {'init':'EPSG:6487'}
ngdf = gpd.GeoDataFrame(data=ngeo, crs=thecrs)
ngdf = ngdf.set_geometry('geometry')

gdf = gpd.GeoDataFrame(neighborhood_guide_df, crs=thecrs)
gdf = gdf.assign(geometry=Polygon())
gdf = gdf.set_geometry('geometry')


print('\nAssigning geometry...')
#neighborhood_guide_df = neighborhood_guide_df.assign(geometry=Polygon())

for thisnh, thisrow in gdf.iterrows():
#    #print(thisnh)
    compns = ngeo['Name'][ngeo['big_neighborhood'] == thisnh].tolist()
    thesegeos = []
    for i in range(0, len(compns)):
        thesegeos.append(ngdf.geometry[ngeo['Name'] == compns[i]].values[0])
    #print(thesegeos)
    onegeo = cascaded_union(thesegeos)
    if (len(compns) > 0):
        #gdf.loc[thisnh, 'geometry'] = onegeo
        gdf.geometry.loc[thisnh] = onegeo

print('backing up...')
gdf_bk = gdf
print('DONE!')

In [ ]:
gdf['nLoans'].sort_values()

In [ ]:
#### DISPLAY FOR ONE STATE AT A TIME

#### DISPLAY BLACK AND WHITE ON TWO SEPARATE MAPS; ALSO SET COORD LIMITS MANUALLY
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(16,8))

ax1 = gdf.plot(ax=ax1, column='percent_black', cmap='Greys', edgecolor='yellow')
ax1.set_title('Percent black (from OpenBaltimore GIS)', fontsize=12)
ax1.set_aspect('equal')

ax2 = gdf.plot(ax=ax2, column='nLoans', cmap='viridis', edgecolor='black')
ax2.set_title('Number of CRA loans originated', fontsize=12)
ax2.set_aspect('equal')

ax1.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
ax2.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

# add colorbar
cax1 = fig.add_axes([0.12, 0.08, 0.36, 0.03])
sm1 = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=12)
cbar1.set_label('Percent black', fontsize=16)

# add colorbar
cax2 = fig.add_axes([0.545, 0.08, 0.36, 0.03])
sm2 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=2300))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, format='.0f', orientation='horizontal')# ticks=np.arange(0, 110, 10), orientation='horizontal')

cax2.tick_params(labelsize=12)
cbar2.set_label('Number of CRA loans', fontsize=16)

#plt.title('Alabama 2016 Presidential election results by congressional district', fontsize=20, position=(-0.05,27.5))
plt.show()
#print('ok')

#gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
#type(ourdata_gdf)
print('Saving...')
fig.savefig('/home/idies/workspace/Storage/raddick/persistent/cra/baltimore_csa.svg', format='svg', dpi=300)
print('DONE!')

In [ ]:
nfile = '/home/idies/workspace/Storage/raddick/persistent/cra/metadata/shapefiles/Neighborhoods.shp'
ngeo = gpd.read_file(nfile)
ngeo = ngeo.set_index('OBJECTID')
ngeo.sample(1)
neighborhoods_list = ngeo.groupby('Name').size().sort_index().index.values.tolist()
with io.open('/home/idies/workspace/Storage/raddick/persistent/cra/metadata/neighborhoods_list.csv', 'w') as f:
    f.write('neighborhood')
    f.write('\n')
    for thisn in neighborhoods_list:
        towrite = thisn + '\n'
        f.write(towrite)
print('DONE!')